In [23]:
import json
import os
import glob
import numpy as np
from datetime import datetime

# Set up paths
analysis_dir = "/home/rcd/Desktop/Workspace/remote-control/analysis"
output_dir = os.path.join(analysis_dir, "29_08_2025", "pre_processed")
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def find_latency_json_files(directory):
    """Find all latency JSON files in the analysis directory"""
    json_files = []

    # Search for JSON files in the analysis directory and subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json') and 'Latency' in file:
                json_files.append(os.path.join(root, file))

    # Also check for the other format (train_*.json)
    pattern = os.path.join(directory, "train_*_latency_*.json")
    json_files.extend(glob.glob(pattern))

    return json_files

json_files = find_latency_json_files(analysis_dir)
for x in json_files:
    print(f"Processing: {os.path.basename(x)}")

Processing: Train_Client(Laptop)_Web_Client(Mobile_BD)_1bdb4d47-a2e0-4148-a721-796cf6ff755b_Latency.json
Processing: Train_Client(Laptop)_Web_Client(Mobile_DE)_5d6a5a85-6566-45b9-8b57-48e886539e0b_Latency.json
Processing: Train_Client(Laptop)_Web_Client(Laptop)_e1c07b43-9e1e-4438-ade8-a320a030e494_Latency.json


In [ ]:
def calculate_statistics(data_list):
    """Calculate count, average, min, max for a list of values"""
    if not data_list:
        return {"count": 0, "avg": 0, "min": 0, "max": 0}

    return {
        "count": len(data_list),
        "avg": np.mean(data_list),
        "min": np.min(data_list),
        "max": np.max(data_list)
    }

def process_telemetry_data(telemetry_latencies, video_latencies=None):
    """Process both telemetry and video latency data and recalculate statistics"""
    # Initialize lists for each transport method
    websocket_data = []
    webtransport_data = []
    mqtt_data = []
    videoframe_data = []

    # Extract data from telemetry entries
    for entry in telemetry_latencies:
        if 'ws' in entry and entry['ws'] is not None:
            websocket_data.append(entry['ws'])
        if 'wt' in entry and entry['wt'] is not None:
            webtransport_data.append(entry['wt'])
        if 'mqtt' in entry and entry['mqtt'] is not None:
            mqtt_data.append(entry['mqtt'])

    # Extract data from video latencies if available
    if video_latencies:
        for entry in video_latencies:
            if 'latency' in entry and entry['latency'] is not None:
                videoframe_data.append(entry['latency'])

    # Calculate statistics for each transport method
    statistics = {
        "websocket": calculate_statistics(websocket_data),
        "webtransport": calculate_statistics(webtransport_data),
        "mqtt": calculate_statistics(mqtt_data),
        "videoframe": calculate_statistics(videoframe_data)
    }

    return statistics

In [26]:
def process_json_file(file_path, output_dir):
    """Process a single JSON file and save the result"""
    try:
        # Load the JSON file
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Check if telemetryLatencies exists
        if 'telemetryLatencies' not in data:
            return False
        
        # Get video latencies if available
        video_latencies = data.get('videoLatencies', None)
        
        # Recalculate statistics
        new_statistics = process_telemetry_data(data['telemetryLatencies'], video_latencies)
        
        # Update the data with new statistics
        data['statistics'] = new_statistics
        data['processingTime'] = datetime.now().isoformat()
        data['originalFile'] = os.path.basename(file_path)
        
        # Generate output filename
        original_filename = os.path.basename(file_path)
        output_filename = f"processed_{original_filename}"
        output_path = os.path.join(output_dir, output_filename)
        
        # Save the processed data
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2)
        
        return True
        
    except Exception as e:
        return False

In [27]:
# Process all JSON files
for json_file in json_files:
    process_json_file(json_file, output_dir)